SD-TSIA211:TP1


Question 3:

Question 3.1:

Given $Ax = b$, we aim to demonstrate that $y(t) = \frac{1}{2} \tilde{x}(t)^T w_2 + \tilde{x}(t)^T w_1 + w_0$.

Given the definition of $A$ and $w$, and the fact that $Ax = b$, we have:

$$
\begin{align*}
\tilde{x}(t)^T w_1 + w_0 &= y(t) - y(t) \times \tilde{x}(t)^T w_2 \\
&= y(t) - \tilde{x}(t)^T (Aw)_2 \\
&= y(t) \text{ for each } t.
\end{align*}
$$

Rearranging this equation:

$$
\tilde{x}(t)^T w_1 + w_0 = y(t) + y(t) \times \tilde{x}(t)^T w_2
$$

Bringing $y(t)$ terms together:

$$
\tilde{x}(t)^T w_1 + w_0 = y(t) (1 + \tilde{x}(t)^T w_2)
$$

And isolating $y(t)$:

$$
y(t) = \frac{\tilde{x}(t)^T w_1 + w_0}{1 + \tilde{x}(t)^T w_2}
$$

This demonstrates that if $Ax = b$, then the relationship $y(t) = \frac{1}{2} \tilde{x}(t)^T w_2 + \tilde{x}(t)^T w_1 + w_0$ is indeed valid.


The dot product $ \mathbf{w}_1^T \tilde{\mathbf{x}}(t) $

- $ \mathbf{w}_1^T $ is the transpose of the weight vector $ \mathbf{w}_1 $, making it a row vector.
- $ \tilde{\mathbf{x}}(t) $ is a column vector of standardized measurements at time t
- The dot product $ \mathbf{w}_1^T \tilde{\mathbf{x}}(t) $ is a scalar, representing a weighted sum of the measurements.

The dot product $ \tilde{\mathbf{x}}(t)^T \mathbf{w}_1 $

- $ \tilde{\mathbf{x}}(t)^T $ is the transpose of $ \tilde{\mathbf{x}}(t) $ making it a row vector.
- $ \mathbf{w}_1 $ is a column vector.
- The dot product $ \tilde{\mathbf{x}}(t)^T \mathbf{w}_1 $ is also a scalar, and due to the properties of transpose and dot product, it's equal to $ \mathbf{w}_1^T \tilde{\mathbf{x}}(t) $

Question 3.2:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2
matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]



# Constructing matrices for the test set
M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]


In [6]:
w, residuals, _, _ = np.linalg.lstsq(A, b, rcond=None)
print("Weights:", w)


Weights: [-0.00927821  0.08309371 -0.03672704 ...  0.01980595 -0.03057174
 -0.01188614]


Question 3.3:

In [7]:
predictions = np.dot(A_test, w)
from sklearn.metrics import mean_squared_error, r2_score


mse = 0.5*mean_squared_error(b_test, predictions)
print(f"R-squared on Test Set: {r2_score(b_test, predictions)}")
print(f"Mean Squared Error on Test Set: {mse}")

R-squared on Test Set: -40.87585582906962
Mean Squared Error on Test Set: 390.44923967618354


Question 3.4:

$$
\min_{x} \frac{1}{2} \|Ax - b\|^2_2 + \frac{\lambda}{2} \|x\|^2_2
$$

Expands to:

$$
\min_{x} \frac{1}{2} (Ax - b)^T(Ax - b) + \frac{\lambda}{2} x^T x
$$

Which simplifies to:

$$
\min_{x} \frac{1}{2} (x^T A^T Ax - x^T A^T b - b^T Ax + b^T b) + \frac{\lambda}{2} x^T x
$$

Modified Form:

$$
\min_{x} \left\| \begin{pmatrix} A \\ \sqrt{\lambda}I \end{pmatrix} x - \begin{pmatrix} b \\ 0 \end{pmatrix} \right\|_2^2
$$

Expands to:

$$
\min_{x} \left( \begin{pmatrix} A \\ \sqrt{\lambda}I \end{pmatrix} x - \begin{pmatrix} b \\ 0 \end{pmatrix} \right)^T \left( \begin{pmatrix} A \\ \sqrt{\lambda}I \end{pmatrix} x - \begin{pmatrix} b \\ 0 \end{pmatrix} \right)
$$

Which simplifies to:

$$
\min_{x} \left( x^T \begin{pmatrix} A^T & \sqrt{\lambda}I \end{pmatrix} \begin{pmatrix} A \\ \sqrt{\lambda}I \end{pmatrix} x - 2x^T \begin{pmatrix} A^T & \sqrt{\lambda}I \end{pmatrix} \begin{pmatrix} b \\ 0 \end{pmatrix} + \begin{pmatrix} b \\ 0 \end{pmatrix}^T \begin{pmatrix} b \\ 0 \end{pmatrix} \right)
$$

This simplifies to:

$$
\min_{x} \left( x^T (A^T A + \lambda I) x - 2x^T A^T b + b^T b \right)
$$

In [10]:
lambda_value = 100.0

A_regularized = np.vstack([A, np.sqrt(lambda_value) * np.eye(A.shape[1])])
b_regularized = np.concatenate([b, np.zeros(A.shape[1])])


w_regularized, _, _, _ = np.linalg.lstsq(A_regularized, b_regularized, rcond=None)


predictions_regularized = np.dot(A_test, w_regularized)
mse_regularized = 0.5*mean_squared_error(b_test, predictions_regularized)

print(f"R-squared on Test Set: {r2_score(b_test, predictions_regularized)}")
print(f"Regularized Mean Squared Error on Test Set: {mse_regularized}")

R-squared on Test Set: -15.144132574526477
Regularized Mean Squared Error on Test Set: 150.5274140470126


It is noticed that the test mean square error with the unregularized one is higher.

Question 3.5:

Given the function $f_1(w) = \frac{1}{2} \| Aw - b \|^2_2 + \frac{\lambda}{2} \| w \|^2_2$, we calculate its gradient as follows:

1. **Gradient of the First Term:** The first term is a least squares term. The gradient of $\frac{1}{2} \| Aw - b \|^2_2$ with respect to $w$ is:

   $$
   \nabla_w \left( \frac{1}{2} \| Aw - b \|^2_2 \right) = A^T (Aw - b)
   $$

2. **Gradient of the Second Term:** The second term is the L2 regularization term. The gradient of $\frac{\lambda}{2} \| w \|^2_2$ with respect to $w$ is:

   $$
   \nabla_w \left( \frac{\lambda}{2} \| w \|^2_2 \right) = \lambda w
   $$

Therefore, the total gradient of $f_1(w)$ is the sum of these two gradients:

$$
\nabla_w f_1(w) = A^T (Aw - b) + \lambda w
$$



The function is convex, sum of two convex functions is convex.

Question 3.6:

Here we will use constant step size, the step size if the inverse of Lipschitz constant:

Given the function $$ f(w) = \frac{1}{2} \| Aw - b \|^2_2 + \frac{\lambda}{2} \| w \|^2_2 $$, we want to find the Lipschitz constant $$ L $$ for the gradient of this function. The gradient is given by:

$$
\nabla f(w) = A^T (Aw - b) + \lambda w.
$$

To find the Lipschitz constant \( L \) for this gradient, we need a constant such that for all \( w_1, w_2 \),

$$
\| \nabla f(w_1) - \nabla f(w_2) \| \leq L \| w_1 - w_2 \|.
$$

Let's calculate \( \nabla f(w_1) - \nabla f(w_2) \):

$$
\nabla f(w_1) - \nabla f(w_2) = (A^T A w_1 + \lambda w_1) - (A^T A w_2 + \lambda w_2) = A^T A (w_1 - w_2) + \lambda (w_1 - w_2).
$$

Now, using the property of matrix norms:

$$
\| A^T A (w_1 - w_2) + \lambda (w_1 - w_2) \| \leq \| A^T A (w_1 - w_2) \| + \| \lambda (w_1 - w_2) \|.
$$

Using the sub-multiplicative property of norms:

$$
\| A^T A (w_1 - w_2) \| \leq \| A^T A \| \| w_1 - w_2 \|, \quad \| \lambda (w_1 - w_2) \| = \lambda \| w_1 - w_2 \|.
$$

Thus,

$$
\| A^T A (w_1 - w_2) + \lambda (w_1 - w_2) \| \leq (\| A^T A \| + \lambda) \| w_1 - w_2 \|.
$$

Therefore, the Lipschitz constant \( L \) is the largest singular value of $A^T A$ (which is the square of the largest singular value of A, or the spectral norm of $ A^T A $ plus $ \lambda $:

$$
L = \| A^T A \| + \lambda.
$$

In [20]:
import time
product_matrix = np.dot(np.transpose(A), A)
eigenvalues, _ = np.linalg.eig(product_matrix)
max_eigenvalue = np.max(np.real(eigenvalues))


Lipschitz_constatnt = max_eigenvalue + lambda_value
step_size = 1 / Lipschitz_constatnt
 
def gradient_f1(A, b, w, lambda_value):
    return np.dot(A.T, np.dot(A, w) - b) + lambda_value * w

w_k = np.zeros(A.shape[1])
start_time = time.time()
iteration = 0
while time.time() - start_time < 30:
    gradient = gradient_f1(A, b, w_k, lambda_value)
    if np.linalg.norm(gradient) <= 1:
        print(f"Converged after {iteration + 1} iterations.")
        iteration += 1
        break
    w_k = w_k - step_size * gradient
    iteration += 1
print(np.linalg.norm(gradient))
print(iteration)
print(w_k)

69.33454875565108
20941
[-0.01425106  0.05605583  0.00349638 ...  0.01356557 -0.04187102
  0.02009826]


Question 3.7:

Here we will use exact line search to calculate the step size:

In [12]:
import numpy as np
from scipy.optimize import minimize_scalar

def ridge_regression_loss(x, A, b, lambda_reg):
    least_squares_loss = 0.5 * np.sum((A @ x - b) ** 2)
    regularization_term = lambda_reg * np.sum(x ** 2)
    return least_squares_loss + regularization_term

def ridge_regression_gradient(x, A, b, lambda_reg):
    gradient = A.T @ (A @ x - b) + 2 * lambda_reg * x
    return gradient

def function_to_minimize(t, x, A, b, lambda_reg):
    grad = ridge_regression_gradient(x, A, b, lambda_reg)
    return ridge_regression_loss(x - t * grad, A, b, lambda_reg)

w_k = np.zeros(A.shape[1])
start_time = time.time()

res = minimize_scalar(function_to_minimize, args=(w_k, A, b, lambda_value))
step_size_ = res.x
iteration = 0
while time.time() - start_time < 30:
    gradient = gradient_f1(A, b, w_k, lambda_value)
    if np.linalg.norm(gradient) <= 1:
        print(f"Converged after {iteration + 1} iterations.")
        iteration += 1
        break
    w_k = w_k - step_size_ * gradient
    iteration += 1
print(iteration)
print(w_k)

20836
[-0.01376004  0.05643976  0.00263156 ...  0.01405717 -0.04075652
  0.01924853]


Question 4:

Question 4.1:

Given the function $$ F_2(w) = \frac{1}{2} \| Aw - b \|^2_2 + \lambda \| w \|_1 $$, we split it into  $f_2 $ and $ g_2 $:

1. **Differentiable Part ($ f_2 $)**: This is the least squares term, which is smooth and differentiable.
   $$
   f_2(w) = \frac{1}{2} \| Aw - b \|^2_2
   $$

2. **Non-Differentiable Part ($ g_2 $)**: This is the L1 regularization term, which is not differentiable at 0 but has a simple proximal operator.
   $$
   g_2(w) = \lambda \| w \|_1
   $$

**Calculating the Gradient of $ f_2 $**

The gradient of $ f_2(w) $ with respect to $ w $ is:
   $$
   \nabla f_2(w) = A^T (Aw - b)
   $$

**Proximal Operator of $ g_2 $**

The proximal operator for the L1 norm (which corresponds to $ g_2(w) = \lambda \| w \|_1 $) is the soft thresholding operator. For a given vector $w $ and a scalar $ t $, the proximal operator $ \text{prox}_{t g_2} $ is defined as:
   $$
   \text{prox}_{t g_2}(w)_i = \text{sign}(w_i) \max(|w_i| - \lambda t, 0)
   $$
for each component $ w_i $ of the vector $ w$.

Question 4.2:

In [32]:
def objective_function(x):
    return 0.5 * np.linalg.norm(b - A @ x) ** 2 +  lambda_reg * np.linalg.norm(x, 1)
def proxi(w, lambda_t):
    return np.sign(w) * np.maximum(np.abs(w) - lambda_t, 0)

def proximal_gradient_descent(A, b, lambda_reg, step_size_lasso, max_iter=100, tol=1e-6):
    w_k_lasso= np.zeros(A.shape[1])
    for i in range(max_iter):
        grad = A.T @ (A @ w_k_lasso - b)
        w_new = proxi(w_k_lasso - step_size_lasso * grad, step_size_lasso * lambda_reg)  
        if np.linalg.norm(w_new - w_k_lasso) < tol:
            break
        w_k_lasso = w_new
    return w_k_lasso

lambda_reg = 200
step_size_lasso = 1 / max_eigenvalue

w_lasso = proximal_gradient_descent(A, b, lambda_reg, step_size_lasso)
min_value1 = objective_function(w_lasso)
print(min_value1)
print(w_lasso)

4092.2498839193945
[-0.          0.00370377  0.00132033 ... -0.          0.00097475
 -0.0008736 ]


Question 4.3:

In [34]:
def f(x):
    return 0.5 * np.linalg.norm(b - A @ x) ** 2
def line_search(x, _grad, _a=1/2, _b=1, _beta = 1/2):
    l = 0
    _gamma = _b*_a**l
    x_plus = x - _gamma*_grad
    while f(x_plus) > f(x) + _beta* (_grad @ (x_plus-x)) :
        l += 1
        _gamma = _b*_a**l
        x_plus = x - _gamma*_grad
    return l

def proximal_gradient_descent(x0, eps = 1e-5, max_iter = 100):
    x = x0
    arr_zeros = np.zeros(x.shape)
    _iter = 0
    rho = 0.02
    norm_arr = []
    a, b , beta = 1/2, 1, 1/2
    grad = grad_f(x)
    norm = np.linalg.norm(grad,2)
    norm_arr.append(norm)
    
    while (norm > eps) & (_iter <= max_iter):
        _iter += 1
        gamma = b * a**line_search(x, grad)
        xi = x - gamma*grad
        x = np.sign(xi)*np.maximum(np.abs(xi)-rho*gamma,arr_zeros)
        grad = grad_f(x)
        norm = np.linalg.norm(grad,2)
        norm_arr.append(norm)

    
    return x

def grad_f(w_k_lasso):
    return A.T @ (A @ w_k_lasso - b)
    

lambda_reg = 200
learning_rate =  1 / max_eigenvalue
w_k_lasso_line = np.zeros(A.shape[1])
w_line_search = proximal_gradient_descent(w_k_lasso_line, 1e-5, 100)
min_value2 = objective_function(w_k_lasso_line)
print(min_value2)
print(w_line_search)


7463.867958769965
[-0.00359126  0.01788695  0.00991634 ...  0.00162506 -0.00429348
 -0.00084256]


Ex 5: 

we do not implement the method but we give some important information that express our understanding here we go :

In our machine learning problem, the primary goal is to find the optimal regularization parameter $\rho$ that achieves a balance between reducing model complexity and ensuring good generalization on new data. This is approached through a bilevel optimization framework:

Inner Level: Minimize a regularized loss function on the training dataset $(X_{\text{train}}, y_{\text{train}})$. The loss function combines a predictive error term (like mean square error) and a regularization term. Two types of regularization are considered: L2 (Ridge, $\frac{1}{2} \|w\|_2^2$) and L1 (Lasso, $\|w\|_1$), where $w$ represents model parameters.

Outer Level: Maximize the model's performance on a separate validation set $(X_{\text{valid}}, y_{\text{valid}})$ by selecting the best $\rho$. The performance is measured using a loss function, typically the mean square error.

Grid Search Method: Due to the complexity of the optimization problem, you employ a grid search to find the best $\rho$. You systematically evaluate a range of $\rho$ values generated by $\{\rho_0 \cdot a^k : k \in \{0,1,\ldots,K\}\}$, where $\rho_0$ is a positive base value, $a$ is a factor less than 1, and $K$ is an integer defining the grid's extent.

Evaluation and Selection: For each $\rho$ in the grid, the model is trained (inner optimization), and its performance is assessed on the validation set. The $\rho$ value (and corresponding model parameters $w$) that leads to the lowest validation loss is selected as the optimal choice.

This approach effectively balances the trade-offs inherent in regularized regression problems, optimizing model complexity for better generalization while also managing the computational demands of the optimization process.


Ex 6:

based on our run that we do and by examining the result using different method we can make a comparison as follow:

**Optimal Weights:**

L1 Regularization tends to produce sparse solutions with some weights exactly zero, enhancing model interpretability through feature selection.

L2 Regularization typically results in non-sparse solutions where weights are reduced but not zeroed, retaining all features but with less interpretability.

**Convergence Speed:**

L1 Regularization generally converges faster, especially with streamlined optimization techniques.

L2 Regularization often requires more iterations to converge, reflecting its continuous, gradual weight reduction.

**Interpretability and Overfitting:**

L1 is preferable for simpler, more interpretable models, useful in scenarios where understanding feature influence is key.

L2, by retaining all features with smaller weights, reduces overfitting risk but at the cost of lower interpretability.

**Algorithm Efficiency:**

The efficiency of L1 can be influenced by the choice of optimization techniques, with some methods accelerating convergence.

L2's slower convergence might impact computational efficiency, especially in larger datasets or complex models.
